In [1]:
import pytz
import datetime
import pandas as pd
import collections
import re
from pymongo import MongoClient
import numpy as np
import pprint

ModuleNotFoundError: No module named 'pymongo'

In [6]:
def removeSpecialCharacters(value):
    return re.sub('[^A-Za-z0-9]+', ' ', str(value))

def cleanDataColumns(data):
    trueCol = []
    for col in data.columns:
        trueCol.append(removeSpecialCharacters(col))
    data.columns = trueCol
    return data

def limpa_cpf(cpfString):
    cpfString = cpfString.replace('.','')
    cpfString = cpfString.replace('-','')
    return int(cpfString)    

def limpeza_de_cpfs(dados, coluna):
    dados[coluna] = dados.loc[:,coluna].apply(limpa_cpf)
    return dados
            
        
def coluna_para_datetime(data, coluna):
    data[coluna] = pd.to_datetime(data[coluna], format='%d/%m/%Y')
    data[coluna] = data.loc[:,coluna].apply(date_to_Brazil_East_Utc)
    return data

def string_to_datetime(datetime_string):
    datetime_string = datetime_string.split('/')
    datetime_value = datetime.datetime(
        int(datetime_string[2]),
        int(datetime_string[1]),
        int(datetime_string[0]))
    return date_to_Brazil_East_Utc(datetime_value)

def date_to_Brazil_East_Utc(datetime_value): # ele adianta o relogio em 3 horas
    if str(datetime_value) == 'NaT':
        datetime_value = datetime.datetime(1500,1,1) # minha data nulla e 01/01/1500
    local = pytz.timezone("Brazil/East")
    local_dt = local.localize(datetime_value, is_dst=True) # esse is_dst e pra ambiguidades, deve ser horario de verao aqui
    utc_dt = local_dt.astimezone(pytz.utc)
    return utc_dt.strftime ("%Y-%m-%d %H:%M:%S")

def float_latin_nan_e_zero(entry):
    if _isNan(entry):
        return 0.0e0
    else:
        return floatLatin(entry)
    

def floatLatin(entry):
    if _isNan(entry):
        return np.nan
    entry = str(entry).replace('.','')
    entry = str(entry).replace(',','.')
    return float(entry)

def _isNan(value):
    if isinstance(value, str):
        return value == 'nan'
    else:
        return np.isnan(value)


# TRANSFORMACOES ESPECIFICAS DA TABELA ACORDOS

def transforma_Assessoria(valor):
    if str(valor) == 'nan':
        return 'SEMEAR'
    else:
        return str(valor)

def transforma_Cobrador(valor):
    if 'Semear_Renego' in valor:
        return 'Renegociacao'
    elif 'Multiparcelas' in valor:
        return 'Multiparcelas'
    else:
        return 'NULO'

# TRANSFORMACOES ESPECIFICAS DA TABELA ACIONAMENTOS

def coluna_para_datetime_hora(data, coluna):
    data[coluna] = pd.to_datetime(data[coluna], format='%d/%m/%Y %H:%M')
    data[coluna] = data.loc[:,coluna].apply(date_to_Brazil_East_Utc)
    return data

# TRANSFORMACOES ESPECIFICAS DO RELATORIO 3040

def coluna_para_datetime_hora_seg(data, coluna):
    data[coluna] = pd.to_datetime(data[coluna], format='%Y-%m-%d %H:%M:%S')
    data[coluna] = data.loc[:,coluna].apply(date_to_Brazil_East_Utc)
    return data



# Leitura dos dados

In [ ]:
# O DATA POOL EXIGE QUE ESSES DADOS SEJAM ENRIQUECIDOS COM A ORIGINACAO DA SAFRA

In [2]:
dados = pd.read_csv('liquidacoes_012019_062019.csv', sep=',', encoding='latin-1', dtype=str)

In [7]:
dados = cleanDataColumns(dados)

In [ ]:
dados = coluna_para_datetime(dados, 'Vencimento')
dados = coluna_para_datetime(dados, 'Liquida o')

In [ ]:
def remover_o_codigo_contrato(contrato):
    return int(contrato.split('/')[0])
dados['Contrato_sem_codigo'] = dados.loc[:,'Contrato'].apply(remover_o_codigo_contrato)

In [8]:
def codigo_de_renegociacao(contrato):
    return int(contrato.split('/')[1])
dados['Contrato_codigo_renegociacao'] = dados.loc[:,'Contrato'].apply(codigo_de_renegociacao)

In [9]:
dados.loc[:,'Parcela'] = dados.loc[:,'Parcela'].astype(int)   # 0 e a entrada da renegociacao

In [10]:
produtos_codigos = []
produtos_descricoes = []
for i in dados.index:
    prod = dados.loc[i,'Produto']
    prod_codigo = int(prod.split(' ')[0])
    prod_descr = prod
    produtos_codigos.append(prod_codigo)
    produtos_descricoes.append(prod_descr)
dados['produtos_codigos'] = produtos_codigos
dados['produtos_descricoes'] = produtos_descricoes # o ideal e adicionar como metadado

In [11]:
dados = limpeza_de_cpfs(dados, 'CPF CNPJ')

In [12]:
dados.loc[:,'Dias'] = dados.loc[:,'Dias'].astype(int)

In [13]:
codigo_da_forma_de_liquidacao = []
descricao_da_forma_de_liquidacao = []
for i in dados.index:
    forma_de_liquidacao = str(dados.loc[i,'Forma'])
    if forma_de_liquidacao == 'nan':
        codigo_da_forma_de_liquidacao.append(-1)
        descricao_da_forma_de_liquidacao.append('NULO')
    else:
        codigo_da_forma_de_liquidacao.append(int(forma_de_liquidacao.split(' ')[0]))
        descricao_da_forma_de_liquidacao.append(forma_de_liquidacao)
dados['codigo_da_forma_de_liquidacao'] = codigo_da_forma_de_liquidacao
dados['descricao_da_forma_de_liquidacao'] = descricao_da_forma_de_liquidacao # o ideal e adicionar como metadado

In [14]:
collections.Counter(dados.loc[:,'Estornado']) # 'NÃ£o'  e 'Sim'

Counter({'NÃ£o': 2185025, 'Sim': 31})

In [15]:
dados.loc[:,'Recebido'] = dados.loc[:,'Recebido'].apply(floatLatin)

# Teste da tabela limpa

In [ ]:
for i in range(5):
    Numero_do_contrato = dados.loc[i,'Contrato_sem_codigo']
    Codigo_renegociacao_contrato = dados.loc[i,'Contrato_codigo_renegociacao']
    Parcela = dados.loc[i, 'Parcela']
    Data_de_vencimento_da_parcela_liquidada = dados.loc[i, 'Vencimento']
    Data_de_liquidacao = dados.loc[i, 'Liquida o']
    Codigo_do_produto = dados.loc[i, 'produtos_codigos']
    Situacao_da_liquidacao = dados.loc[i,'Situa o']
    Numero_do_CPF = dados.loc[i,'CPF CNPJ']
    Dias_em_atraso_na_hora_da_liquidacao = dados.loc[i,'Dias']
    Codigo_da_forma_de_liquidacao = dados.loc[i,'codigo_da_forma_de_liquidacao']
    Estornado = dados.loc[i,'Estornado']
    Valor_recebido_na_liquidacao = dados.loc[i,'Recebido']
    liquidacao_dict = {
        'Numero_do_contrato': Numero_do_contrato,
        'Data_da_liquidacao': Data_de_liquidacao,
        'Codigo_renegociacao_contrato': Codigo_renegociacao_contrato,
        'Parcela_liquidada': Parcela,
        'Data_de_vencimento_da_parcela_liquidada': Data_de_vencimento_da_parcela_liquidada,
        'Codigo_do_produto': Codigo_do_produto,
        'Situacao_da_liquidacao': Situacao_da_liquidacao, # poucos casos com situacao parcial aqui
        'Numero_do_CPF': Numero_do_CPF,
        'Dias_em_atraso_na_hora_da_liquidacao': Dias_em_atraso_na_hora_da_liquidacao,
        'Codigo_da_forma_de_liquidacao': Codigo_da_forma_de_liquidacao,
        'Descricao_da_forma_de_liquidacao': Descricao_da_forma_de_liquidacao,
        'Estornado': Estornado,
        'Valor_recebido_na_liquidacao': Valor_recebido_na_liquidacao
    }
    print(liquidacao_dict)
    
    

In [ ]:
dados.columns

# Definicao da coluna liquidacoes

In [ ]:
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client.cobrancas
liquidacoes_mongo = db.liquidacoes

# Carga no banco de dados

In [ ]:
# demora mais ou menos 1 hora essa carga aqui
liquidacao_ID = 0 
for i in dados.index:
    liquidacao_ID += 1
    Numero_do_contrato = dados.loc[i,'Contrato_sem_codigo']
    Codigo_renegociacao_contrato = dados.loc[i,'Contrato_codigo_renegociacao']
    Parcela = dados.loc[i, 'Parcela']
    Data_de_vencimento_da_parcela_liquidada = dados.loc[i, 'Vencimento']
    Data_de_liquidacao = dados.loc[i, 'Liquida o']
    Codigo_do_produto = dados.loc[i, 'produtos_codigos']
    Situacao_da_liquidacao = dados.loc[i,'Situa o']
    Numero_do_CPF = dados.loc[i,'CPF CNPJ']
    Dias_em_atraso_na_hora_da_liquidacao = dados.loc[i,'Dias']
    Codigo_da_forma_de_liquidacao = dados.loc[i,'codigo_da_forma_de_liquidacao']
    Estornado = dados.loc[i,'Estornado']
    Valor_recebido_na_liquidacao = dados.loc[i,'Recebido']
    dados_dict = {
        'Liquidacao_ID': liquidacao_ID,
        'Numero_do_contrato': int(Numero_do_contrato),
        'Data_da_liquidacao': Data_de_liquidacao,
        'Codigo_renegociacao_contrato': int(Codigo_renegociacao_contrato),
        'Parcela_liquidada': int(Parcela),
        'Data_de_vencimento_da_parcela_liquidada': Data_de_vencimento_da_parcela_liquidada,
        'Codigo_do_produto': int(Codigo_do_produto),
        'Situacao_da_liquidacao': str(Situacao_da_liquidacao), # poucos casos com situacao parcial aqui
        'Numero_do_CPF': int(Numero_do_CPF),
        'Dias_em_atraso_na_hora_da_liquidacao': int(Dias_em_atraso_na_hora_da_liquidacao),
        'Codigo_da_forma_de_liquidacao': int(Codigo_da_forma_de_liquidacao),
        'Estornado': str(Estornado),
        'Valor_recebido_na_liquidacao': Valor_recebido_na_liquidacao
    }

    liquidacoes_mongo.insert_one(dados_dict)

# Metadados que mapeiam os codigos dos produtos

In [ ]:
informacoes_adicionais = db.informacoes_adicionais

In [ ]:
#informacoes_adicionais.insert_one() Adicionar metadados aqui.

# Todos os registros para um dado CPF

In [ ]:
import pprint
cursor = liquidacoes_mongo.find({'Numero_do_CPF': 60014075920})
for document in cursor:
    pprint.pprint(document)

# Todos os registros em um determinado intervalo de datas

In [ ]:
start = date_to_Brazil_East_Utc(datetime.datetime(2019, 6, 3))
end = date_to_Brazil_East_Utc(datetime.datetime(2019, 6, 4))
query_datas_limite = {'Data_da_liquidacao': {'$lt': end, '$gte': start}}
cursor = liquidacoes_mongo.find(query_datas_limite)
for document in cursor:
    print(document)

# ENRIQUECIMENTO DAS LIQUIDACOES COM DATA DE ORIGINACAO

In [ ]:
dados = pd.read_csv('liquidacoes_012019_062019.csv', sep=',', encoding='latin-1', dtype=str)
dados = cleanDataColumns(dados)
dados = limpeza_de_cpfs(dados, 'CPF CNPJ')

In [ ]:
def remover_o_codigo_contrato(contrato):
    return int(contrato.split('/')[0])
dados['Contrato_sem_codigo'] = dados.loc[:,'Contrato'].apply(remover_o_codigo_contrato)

In [ ]:
contratos_liquid = list(set(dados.loc[:,'Contrato_sem_codigo']))

In [ ]:
def get_DatInicio(pdd_table, contrato):
    pdd_x = pdd_table[pdd_table.loc[:,'Contrato'] == contrato].copy()
    pdd_x = pdd_x.reset_index(drop=True)
    return pdd_x.loc[0,'DatInicio']

In [ ]:
# OBTENCAO DAS DATAS DE ORIGINACAO DE TODOS OS PDDs
# (ESSE PASSO DEMORA ALGUMAS HORAS)
data_de_originacao_dos_contratos = {}
pdd_files = ['3040_01_2019.xlsx','3040_02_2019.xlsx','3040_03_2019.xlsx','3040_04_2019.xlsx','3040_05_2019.xlsx','3040_06_2019.xlsx']
for pdd_file in pdd_files:
    pdd_i = pd.read_excel(pdd_file, encoding='latin-1', dtype=str) # demora uns 15min essa merda
    contratos_i = list(set(pdd_i.loc[:,'Contrato']))
    for contrato in contratos_i:
        if contrato in data_de_originacao_dos_contratos:
            continue
        data_de_originacao_dos_contratos[contrato] = get_DatInicio(pdd_i, contrato) 

In [ ]:
contratos_pdd = [int(x) for x in data_de_originacao_dos_contratos.keys()]

In [ ]:
diff = set(contratos_liquid) - set(contratos_pdd)

In [ ]:
orig_data = []
for key in data_de_originacao_dos_contratos:
    orig_data.append([key, data_de_originacao_dos_contratos[key]])
pd.DataFrame(orig_data).to_csv('Contratos_originacao_tirado_do_pdd.csv',sep=';')

In [16]:
dados_originacao = pd.read_csv('Contratos_originacao_tirado_do_pdd.csv', sep = ';')
dados_originacao.iloc[:,1] = dados_originacao.iloc[:,1].astype(int)
data_de_originacao_dos_contratos = {}
for i in dados_originacao.index:
    contrato = dados_originacao.iloc[i,1]
    data_origem = datetime.datetime.strptime(dados_originacao.iloc[i,2], "%Y-%m-%d %H:%M:%S")
    data_origem = date_to_Brazil_East_Utc(data_origem)
    data_de_originacao_dos_contratos[contrato] = data_origem

# TODOS OS REGISTROS PARA UM DADO NUMERO DE CONTRATO

In [23]:
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client.cobrancas
liquidacoes_mongo = db.liquidacoes

In [ ]:
# EXEMPLO DE QUERY
cursor = liquidacoes_mongo.find({'Numero_do_contrato': 54578134})
for document in cursor:
    pprint.pprint(document)
    print(document['Liquidacao_ID'])

# ENRIQUECIMENTO DO RELATORIO DE LIQUIDACOES COM DATA DE ORIGINACAO (PDD)

In [34]:
#def adicione_data_de_originacao(liquidacao_ID, data_de_originacao, liquidacoes_mongo):
#    myquery = {'Liquidacao_ID':liquidacao_ID}
#    newvalues = { "$set": { "Data_de_originacao_do_contrato": data_de_originacao}}
#    liquidacoes_mongo.update_one(myquery, newvalues)

In [ ]:
# CARGA NO BANCO DE DADOS DESSE JEITO DEMORA MAIS DE DIA
#for contrato in data_de_originacao_dos_contratos:
#    data_de_originacao = data_de_originacao_dos_contratos[numero_do_contrato]
#    cursor = liquidacoes_mongo.find({'Numero_do_contrato': int(contrato)})
#    for document in cursor:
#        liquidacao_ID = document['Liquidacao_ID']
#        adicione_data_de_originacao(liquidacao_ID, data_de_originacao, liquidacoes_mongo)

In [41]:
originacoes = []
for i in dados.index:
    numero_contrato = dados.loc[i,'Contrato_sem_codigo']
    if numero_contrato in data_de_originacao_dos_contratos:
        datetime_origin = datetime.datetime.strptime(data_de_originacao_dos_contratos[numero_contrato], '%Y-%m-%d %H:%M:%S')
        originacoes.append(datetime_origin)
    else:
        datetime_origin = datetime.datetime.strptime('1500-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
        originacoes.append(datetime_origin)

In [42]:
dados['Data_de_originacao'] = originacoes

In [ ]:
dados_validos = dados[dados['Data_de_originacao'] > datetime.datetime(1600,1,1)].copy() # removendo casos onde a data de originicao nao foi encontrada

# ENRIQUECIMENTO DO RELATORIO DE LIQUIDACOES COM OCORRENCIAS DE CONTATOS TELEFONICOS

In [78]:
dados_cobrancas = dados_validos[dados_validos.loc[:,'Dias'] > 4].copy()

In [81]:
dados_cobrancas = dados_cobrancas.reset_index(drop=True)

In [104]:
acionamentos = pd.read_csv('Acionamentos_01012019_a_30062019.csv', sep=';', encoding='latin-1', dtype=str)
acionamentos = cleanDataColumns(acionamentos)
cpf_null = acionamentos[acionamentos.loc[:,'CPF CNPJ'].isnull()].index
acionamentos = acionamentos.drop(cpf_null)
acionamentos = acionamentos.reset_index(drop=True)
acionamentos = limpeza_de_cpfs(acionamentos, 'CPF CNPJ')
# Criacao da coluna datahora
datahora = []
for i in acionamentos.index:
    datahora.append(acionamentos.loc[i,'Data'] + ' ' + acionamentos.loc[i,'Hor rio'])
acionamentos['datahora'] = datahora
acionamentos = coluna_para_datetime_hora(acionamentos, 'datahora')

In [123]:
# NUMERO DE LIQUIDACOES QUE FORAM PAGAS COM ATRASO >= 5 dias.
# NUMERO DE LIQUIDACOES NESTE PERIODO = 2185056
# NUMERO DE LIQUIDACOES EM ATRASO = 574980  (26%)
# NUMERO DE ACIONAMENTOS NO PERIODO = 139197
# NUMERO QUE NAO ACHEI NO PDD = 109467 (provavelmente a ultima parcela foi paga)
len(dados)
len(dados_cobrancas)
len(acionamentos)

574980

In [254]:
acionamentos.loc[:,'Dias atraso'] = acionamentos.loc[:,'Dias atraso'].astype(int)

In [256]:
len(acionamentos[acionamentos.loc[:,'Dias atraso'] > 60])

104777

In [263]:
len(acionamentos[
    (acionamentos.loc[:,'Dias atraso'] <= 60) &
    (acionamentos.loc[:,'Dias atraso'] >= 30)])

6905

In [261]:
len(acionamentos[
    (acionamentos.loc[:,'Dias atraso'] < 30)])

27515

In [ ]:
liquidacao_cobrada = []
por_cento = int(len(dados_cobrancas)/100)
for i in dados_cobrancas.index:
    if i % por_cento == 0:
        print(i/por_cento, '%')
        
    cpf = dados_cobrancas.loc[i,'CPF CNPJ']
    data_liq = datetime.datetime.strptime(
        dados_cobrancas.loc[i,'Liquida o'], 
        '%Y-%m-%d %H:%M:%S')    
    dias_em_atraso = dados.loc[i, 'Dias']
    foi_cobrado = False
    acionamento_I = acionamentos[acionamentos.loc[:,'CPF CNPJ'] == cpf].copy()
    if len(acionamento_I) != 0:
        for j in acionamento_I.index:
            data_do_acionamento = acionamento_I.loc[j,'datahora']
            data_do_acionamento = datetime.datetime.strptime(
                data_do_acionamento, 
                '%Y-%m-%d %H:%M:%S')
            dias_que_foi_cobrado = (data_do_acionamento - data_liq).days
            if dias_que_foi_cobrado <= 0 and dias_que_foi_cobrado >= -15:
                foi_cobrado = True
                break
    liquidacao_cobrada.append(foi_cobrado)

In [153]:
collections.Counter(liquidacao_cobrada)

Counter({False: 556584, True: 18396})

In [163]:
dados_cobrancas['foi_cobrada'] = liquidacao_cobrada

In [195]:
dados_cobrancas['Liquida o'] = pd.to_datetime(dados_cobrancas['Liquida o'], format='%Y-%m-%d %H:%M:%S')

# CORTES NAS DATAS DE ORIGINACAO

In [198]:
data_fim = datetime.datetime.strptime('2017-12-31 23:59:59', '%Y-%m-%d %H:%M:%S')
safra_2017_mais = dados_cobrancas[dados_cobrancas.loc[:,'Data_de_originacao'] < data_fim].copy()

In [199]:
data_inicio = datetime.datetime.strptime('2018-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
data_fim = datetime.datetime.strptime('2018-03-31 23:59:59', '%Y-%m-%d %H:%M:%S')
safra_2018_jan_mar = dados_cobrancas[
    (dados_cobrancas.loc[:,'Data_de_originacao'] < data_fim) &
    (dados_cobrancas.loc[:,'Data_de_originacao'] > data_inicio)].copy()

In [200]:
data_inicio = datetime.datetime.strptime('2018-04-01 00:00:00', '%Y-%m-%d %H:%M:%S')
data_fim = datetime.datetime.strptime('2018-06-30 23:59:59', '%Y-%m-%d %H:%M:%S')
safra_2018_abr_jun = dados_cobrancas[
    (dados_cobrancas.loc[:,'Data_de_originacao'] < data_fim) &
    (dados_cobrancas.loc[:,'Data_de_originacao'] > data_inicio)].copy()

In [201]:
data_inicio = datetime.datetime.strptime('2018-07-01 00:00:00', '%Y-%m-%d %H:%M:%S')
data_fim = datetime.datetime.strptime('2018-09-30 23:59:59', '%Y-%m-%d %H:%M:%S')
safra_2018_jul_set = dados_cobrancas[
    (dados_cobrancas.loc[:,'Data_de_originacao'] < data_fim) &
    (dados_cobrancas.loc[:,'Data_de_originacao'] > data_inicio)].copy() 

In [202]:
data_inicio = datetime.datetime.strptime('2018-10-01 00:00:00', '%Y-%m-%d %H:%M:%S')
data_fim = datetime.datetime.strptime('2018-12-31 23:59:59', '%Y-%m-%d %H:%M:%S')
safra_2018_out_dez = dados_cobrancas[
    (dados_cobrancas.loc[:,'Data_de_originacao'] < data_fim) &
    (dados_cobrancas.loc[:,'Data_de_originacao'] > data_inicio)].copy()

In [203]:
data_inicio = datetime.datetime.strptime('2019-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
data_fim = datetime.datetime.strptime('2019-03-31 23:59:59', '%Y-%m-%d %H:%M:%S')
safra_2019_jan_mar = dados_cobrancas[
    (dados_cobrancas.loc[:,'Data_de_originacao'] < data_fim) &
    (dados_cobrancas.loc[:,'Data_de_originacao'] > data_inicio)].copy()

In [204]:
data_inicio = datetime.datetime.strptime('2019-04-01 00:00:00', '%Y-%m-%d %H:%M:%S')
data_fim = datetime.datetime.strptime('2019-06-30 23:59:59', '%Y-%m-%d %H:%M:%S')
safra_2019_abr_jun = dados_cobrancas[
    (dados_cobrancas.loc[:,'Data_de_originacao'] < data_fim) &
    (dados_cobrancas.loc[:,'Data_de_originacao'] > data_inicio)].copy()

In [190]:
len(safra_2017_mais) + len(safra_2018_jan_mar) + len(safra_2018_abr_jun) + len(safra_2018_jul_set) + len(safra_2018_out_dez) + len(safra_2019_jan_mar) + len(safra_2019_abr_jun)

574980

In [248]:
def corte_na_BAD(safra, datas_limite):
    data_inicio, data_fim = datas_limite
    BAD_liquidacao = safra[
        (safra.loc[:,'Liquida o'] < data_fim) &
        (safra.loc[:,'Liquida o'] > data_inicio)].copy()
    cobrancas_com_sucesso = len(BAD_liquidacao[BAD_liquidacao.foi_cobrada == True])
    total = len(BAD_liquidacao)
    if total == 0:
        return 'X'
    return int(100*(total-cobrancas_com_sucesso)/total)

In [219]:
bads = [
    [datetime.datetime.strptime('2019-01-01 00:00:00', '%Y-%m-%d %H:%M:%S'),
     datetime.datetime.strptime('2019-01-31 23:59:59', '%Y-%m-%d %H:%M:%S')],
    [datetime.datetime.strptime('2019-02-01 00:00:00', '%Y-%m-%d %H:%M:%S'),
     datetime.datetime.strptime('2019-02-28 23:59:59', '%Y-%m-%d %H:%M:%S')],
    [datetime.datetime.strptime('2019-03-01 00:00:00', '%Y-%m-%d %H:%M:%S'),
     datetime.datetime.strptime('2019-03-31 23:59:59', '%Y-%m-%d %H:%M:%S')],
    [datetime.datetime.strptime('2019-04-01 00:00:00', '%Y-%m-%d %H:%M:%S'),
     datetime.datetime.strptime('2019-04-30 23:59:59', '%Y-%m-%d %H:%M:%S')],
    [datetime.datetime.strptime('2019-05-01 00:00:00', '%Y-%m-%d %H:%M:%S'),
     datetime.datetime.strptime('2019-05-31 23:59:59', '%Y-%m-%d %H:%M:%S')],
    [datetime.datetime.strptime('2019-06-01 00:00:00', '%Y-%m-%d %H:%M:%S'),
     datetime.datetime.strptime('2019-06-30 23:59:59', '%Y-%m-%d %H:%M:%S')]
]

In [249]:
safra = safra_2019_abr_jun
print(len(safra))
for i in range(6):
    print(corte_na_BAD(safra, bads[i]))

7319
X
X
X
100
94
95


In [10]:
collections.Counter(dados.loc[:,'Situa o'])

Counter({'Liquidado': 2184883, 'Parcial': 165, 'Aberto': 8})

In [11]:
dados[dados.loc[:,'Situa o'] == 'Parcial'].to_csv('exemplo_liquidacao_parcial.csv',sep=';')

In [12]:
dados[dados.loc[:,'Situa o'] == 'Aberto'].to_csv('exemplo_liquidacao_aberta.csv',sep=';')

In [16]:
dados[dados.loc[:,'Estornado'] == 'Sim'].to_csv('exemplo_estornado_sim.csv',sep=';')